# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    exp_return = 0
    total_invested = P['Amount Invested'].sum()
    for i in range(len(P)):
        weight = P.at[i, 'Amount Invested']/total_invested
        exp_return += (rf + P.at[i, 'Beta'] * mkt_rp) * weight
    
    return exp_return
# expected_return(portfolio, 0.02, 0.05)

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    # Write your code here
    total_invested = P['Amount Invested'].sum()
    var = 0
    for i in range(len(P)):
        weight = P.at[i, 'Amount Invested']/total_invested
        var += (weight**2) * (P.at[i, 'Volatility']**2)
    for i in range(len(P) - 1):
        weight1 = P.at[i, 'Amount Invested']/total_invested
        vol1 = P.at[i, 'Volatility']
        for j in range (i + 1, len(P)):
            weight2 = P.at[j, 'Amount Invested']/total_invested
            vol2 = P.at[j, 'Volatility']
            var += 2 * weight1 * weight2 * vol1 * vol2 * cor
    return var ** (1/2)
# volatility(portfolio, -0.2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    risky_asset = vol / mkt_vol
    exp_return = rf + risky_asset * mkt_rp
    return exp_return
# expected_return2(0.02, 0.05, 0.16, 0.2408)

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [2]:
def CAPM_beta(return_path, factors_path):
    return_df = pd.read_csv(return_path)
    factors_df = pd.read_csv(factors_path)
    
    # Extract 'year' and 'month' from the date column in factors_df
    factors_df['year'] = factors_df['date'] // 100
    factors_df['month'] = factors_df['date'] % 100
    
    # Merge return and factors data on 'year' and 'month'
    merged_df = pd.merge(return_df, factors_df[['year', 'month', 'Mkt-RF', 'RF']], on=['year', 'month'], how='inner')
    
    # Calculate excess return for each stock (R_i - Rf)
    merged_df['Excess_Return'] = merged_df['RET'] - merged_df['RF'] / 100  # Convert RF to decimal
    
    # Calculate excess market return (R_mkt - Rf)
    merged_df['Excess_Market_Return'] = merged_df['Mkt-RF'] / 100  # Convert Mkt-RF to decimal
    
    # Calculate Cov(R_i-Rf, R_mkt-Rf) and Var(R_mkt-Rf) for each CUSIP
    cov_var_df = merged_df.groupby('CUSIP').apply(lambda x: pd.Series({
        'BETA': np.cov(x['Excess_Return'], x['Excess_Market_Return'])[0, 1] / 
                                      x['Excess_Market_Return'].var(),
        'RET': x['RET'].mean()
    }))
    
    # Rename the mean of RET to Expected_Return
    cov_var_df.rename(columns={'RET': 'Expected_Return'}, inplace=True)
    
    # Select relevant columns
    result_df = cov_var_df[['Expected_Return', 'BETA']].reset_index()
    
    # Prepare the output DataFrame
    output = pd.DataFrame(data={
        "CUSIP": result_df['CUSIP'],
        "EXP_RETURN": result_df['Expected_Return'],
        "BETA": result_df['BETA'],
    })
    
    return output
return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data
# CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv")

CUSIP  EXP_RETURN       BETA
0  36020   -0.032023  -0.483703
1  36021    0.028259  -1.124363
2  36022    0.048830  -5.149374
3  36023    0.089881  10.187062

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    coefficients = np.polyfit(data['BETA'], data['EXP_RETURN'], deg=1)
    sml_slope = coefficients[1]
    return sml_slope
# slope(CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv"))